In [2]:
import pandas as pd

In [8]:
df = pd.read_csv("raw_data/21.5개시도_유흥업소_단란주점현황.csv")
df

,license_date,open_gbn,type,surroundings,lon,lat
0,19951012,영업/정상,단란주점,NaN,127.300127,36.601953
1,20181127,영업/정상,단란주점,NaN,127.301023,36.601199
2,20021121,영업/정상,단란주점,유흥업소밀집지역,127.300940,36.601856
3,20020409,영업/정상,단란주점,유흥업소밀집지역,127.299671,36.602782
4,19990804,영업/정상,단란주점,학교정화(상대),127.205025,36.678746
...,...,...,...,...,...,...
7114,20001116,영업/정상,룸살롱,유흥업소밀집지역,126.930536,37.610393
7115,20010307,영업/정상,룸살롱,유흥업소밀집지역,126.930733,37.610163
7116,20170630,영업/정상,룸살롱,유흥업소밀집지역,126.929299,37.610537
7117,20171228,영업/정상,룸살롱,유흥업소밀집지역,126.919550,37.620091


In [5]:
df["surroundings"].unique()

array([nan, '유흥업소밀집지역', '학교정화(상대)', '기타', '학교정화(절대)', '주택가주변', '결혼예식장주변',
       '아파트지역'], dtype=object)

In [7]:
df["type"].unique()

array(['단란주점', '노래클럽', '룸살롱', '간이주점', '스텐드바', '기타', '카바레', '고고(디스코)클럽',
       '비어(바)살롱', '관광호텔나이트(디스코)', '요정', nan, '관광호텔나이트(카바레)', '한식', '극장식당'],
      dtype=object)

In [9]:
df["open_gbn"].unique()

array(['영업/정상'], dtype=object)

In [16]:
df["관할서"]="모름"

## 2. 위도경도 데이터로 관할서 지정

In [10]:
# 유흥업소 위도 경도 zip
location = list(zip(df["lon"], df["lat"]))

In [11]:
# geopandas, Point  라이브러리
import geopandas as gpd
from shapely.geometry import Point, Polygon

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [12]:
#위도 경도 데이터를 Point함수를 이용해서 변수로 만듦
#이를 리스트에 넣어줌
st_list = []
for i in location:
    test_code = Point(i)
    st_list.append(test_code)

In [13]:
#관할서 경계데이터 geopandas로 읽어오기
df_js = gpd.read_file('raw_data/1.5개시도_경찰서_관할경계.geojson', encoding='cp949')
df_js.head()

,NAME,PNAME,geometry
0,세종경찰서,충남청,"MULTIPOLYGON (((127.17202 36.73106, 127.17202 ..."
1,진주경찰서,경남청,"MULTIPOLYGON (((128.26697 35.12927, 128.26697 ..."
2,창원서부경찰서,경남청,"MULTIPOLYGON (((128.63363 35.22152, 128.63357 ..."
3,창원중부경찰서,경남청,"MULTIPOLYGON (((128.60966 35.15093, 128.60956 ..."
4,마산동부경찰서,경남청,"MULTIPOLYGON (((128.62696 35.21714, 128.62695 ..."


In [14]:
# 빠른 반복문 진행을 위해 변수지정
df_js_list = df_js["geometry"]
df_js_name = df_js["NAME"]

In [15]:
from tqdm import tqdm

In [17]:
# 위도경도 데이터가 관할경계데이터에 포함되면  관할서의 이름값을 df관할서의 "모름"을 관할서 이름으로 바꿔라 
# 넘 오래걸림...코드 디벨롶 요망

for z in tqdm(range(41)):
    for i in range(7119):
        if st_list[i].within(df_js_list[z]):
            df.iloc[i,-1] = df.iloc[i,-1].replace("모름",df_js_name[z])
            
        else:
            pass

100%|██████████| 41/41 [04:06<00:00,  6.01s/it]


In [19]:
df.to_csv("데이터별_관할서리스트/유흥업소_관할서리스트")

## 3. 유흥업소 데이터 재 전처리

In [3]:
df = pd.read_csv("데이터별_관할서리스트/유흥업소_관할서리스트", index_col = 0)
df

,license_date,open_gbn,type,surroundings,lon,lat,관할서
0,19951012,영업/정상,단란주점,NaN,127.300127,36.601953,세종경찰서
1,20181127,영업/정상,단란주점,NaN,127.301023,36.601199,세종경찰서
2,20021121,영업/정상,단란주점,유흥업소밀집지역,127.300940,36.601856,세종경찰서
3,20020409,영업/정상,단란주점,유흥업소밀집지역,127.299671,36.602782,세종경찰서
4,19990804,영업/정상,단란주점,학교정화(상대),127.205025,36.678746,세종경찰서
...,...,...,...,...,...,...,...
7114,20001116,영업/정상,룸살롱,유흥업소밀집지역,126.930536,37.610393,서울은평경찰서
7115,20010307,영업/정상,룸살롱,유흥업소밀집지역,126.930733,37.610163,서울은평경찰서
7116,20170630,영업/정상,룸살롱,유흥업소밀집지역,126.929299,37.610537,서울은평경찰서
7117,20171228,영업/정상,룸살롱,유흥업소밀집지역,126.919550,37.620091,서울은평경찰서


In [5]:
df.isnull().sum()

license_date       0
open_gbn           0
type               1
surroundings    3594
lon                0
lat                0
관할서                0
dtype: int64

In [9]:
df_ent = df.groupby("관할서", sort = True).count()["license_date"]
df_ent = pd.DataFrame(df_ent).reset_index()
df_ent

,관할서,license_date
0,마산동부경찰서,337
1,마산중부경찰서,487
2,서울강남경찰서,297
3,서울강동경찰서,213
4,서울강북경찰서,184
5,서울강서경찰서,289
6,서울관악경찰서,272
7,서울광진경찰서,93
8,서울구로경찰서,107
9,서울금천경찰서,118


In [10]:
# 관할서명  수정
df_ent["관할서"] = df_ent["관할서"].str.replace("마산", "경남마산")                                     
df_ent["관할서"] = df_ent["관할서"].str.replace("창원", "경남창원")
df_ent["관할서"] = df_ent["관할서"].str.replace("수원", "경기수원")
df_ent["관할서"] = df_ent["관할서"].str.replace("진주", "경남진주")
df_ent["관할서"] = df_ent["관할서"].str.replace("진해", "경남진해")
df_ent["관할서"] = df_ent["관할서"].str.replace("세종", "충남세종")                                           

In [21]:
# 컬럼명 변경
df_ent.rename(columns = {"license_date":"ent_num"}, inplace=True)

In [23]:
# sort 후 파일저장
df_ent.sort_values("관할서").reset_index(drop=True).to_csv("전처리완성본/관할서별_유흥업소 개수")
